<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Linear Regression
    </div>
<br><br>
<font size=4>
   **Name**: Amir hosein Haji mohammad rezaie<br>
   **Student ID**: 99109252<br> <br>

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 1: Linear Regression Model (40 + 30 optional points)
According to <a href="https://github.com/asharifiz/Introduction_to_Machine_Learning/blob/main/Jupyter_Notebooks/Chapter_02_Classical_Models/Linear%20regression.ipynb"><font face="Roboto">Linear Regression Notebook</font></a>, train a linear regression model on an arbitrary dataset. Explain your chosen dataset and split your data into train and test sets, then predict values for the test set using your trained model. Try to find the best hyperparameters for your model. (Using Lasso Regression, Ridge Regression or Elastic Net and comparing them will have extra optional points)
<br> Explain each step of your workflow.

In [1]:
import pandas as pd
import numpy as np
import sklearn
import math
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In this notebook, we use Fish market dataset. This dataset contains informations such as weight, width, height, diagonal length, .... You can find more information about this dataset [here](https://www.kaggle.com/datasets/aungpyaeap/fish-market)

In [2]:
df = pd.read_csv('Fish.csv')
df.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [3]:
df.tail()

,Species,Weight,Length1,Length2,Length3,Height,Width
154,Smelt,12.2,11.5,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,11.7,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,12.1,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,13.2,14.3,15.2,2.8728,2.0672
158,Smelt,19.9,13.8,15.0,16.2,2.9322,1.8792


here we want to predict weight of a fish by giving parameters such as width, height, Length1, ... to out linear regression model to predict weight. so first we split weight column from other columns in out data frame.

In [4]:
y = df.iloc[:,1]
X = df.iloc[:,2:]
y

0      242.0
1      290.0
2      340.0
3      363.0
4      430.0
       ...  
154     12.2
155     13.4
156     12.2
157     19.7
158     19.9
Name: Weight, Length: 159, dtype: float64

X is the input parameters to model and y is the output results (weight results) that we get from model.

In [5]:
X

,Length1,Length2,Length3,Height,Width
0,23.2,25.4,30.0,11.5200,4.0200
1,24.0,26.3,31.2,12.4800,4.3056
2,23.9,26.5,31.1,12.3778,4.6961
3,26.3,29.0,33.5,12.7300,4.4555
4,26.5,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...
154,11.5,12.2,13.4,2.0904,1.3936
155,11.7,12.4,13.5,2.4300,1.2690
156,12.1,13.0,13.8,2.2770,1.2558
157,13.2,14.3,15.2,2.8728,2.0672


Here we split our dataset to train and test sets with ratio of 7/3

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)

In [7]:
X_train

,Length1,Length2,Length3,Height,Width
53,25.0,27.0,30.6,8.5680,4.7736
117,36.5,39.0,41.4,11.1366,6.0030
136,40.1,43.0,45.8,7.7860,5.1296
17,30.4,33.0,38.5,14.9380,5.1975
77,16.2,18.0,19.2,5.2224,3.3216
...,...,...,...,...,...
133,36.0,38.5,41.0,6.3960,3.9770
137,42.0,45.0,48.0,6.9600,4.8960
72,7.5,8.4,8.8,2.1120,1.4080
140,48.3,51.7,55.1,8.9262,6.1712


In [8]:
y_train

53     272.0
117    650.0
136    540.0
17     700.0
77     100.0
       ...  
133    345.0
137    500.0
72       5.9
140    950.0
37      78.0
Name: Weight, Length: 111, dtype: float64

and here we convert data frames to numpy array

In [9]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

here we add one column of ones to X matrix, so we can predict bias for equations in solved W matrix.

In [10]:
x_train = np.append(np.ones((len(X_train), 1)), X_train, axis=1)
x_test = np.append(np.ones((len(X_test), 1)), X_test, axis=1)

## 1. closed from linear regression
first, we get the results from a closed linear regression model that it's loss error function is:
$$\lVert y - Xw \rVert_2^2$$
and the solution for w is:

$$w =(x^T x)^{-1} x^T y$$

so we obtain weight matrix (w) and we use it to predict weights in test set. we obtain the error of prediction by using mean square error:
$$error = \frac{1}{n} \sum_{i=1}^{n}(y_i - X_iw)^2$$

In [11]:
W = np.dot((np.linalg.inv(np.dot(x_train.T, x_train))), np.dot(x_train.T, y_train))
predict = np.dot(x_test, W)
diff = predict - y_test
error = np.sum(diff * diff)/len(diff)
error

16062.711813979142

so in the above code, you can see that we find W matrix and we predict weights of the fish in test set and we obtain the MSE error that is equal to 16062.71 which is a high number and it's not good enough for predicting.

## 2. Ridge regresson
here we use ridge regression for our model to predict the weights values. we use the scikit learn Ridge package and we test our model on different alpha values and we obtain the model with best accuracy (minimum mse error) on test set. we know that ridge regression loss function is:

$$L(w) = \frac{1}{n}\sum_{i=1}^{n}(y+i - X_iw)^2 + alpha * \sum_{j=1}^{m}w_j^2$$

In [12]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error as mse
best_model = None
min_error = math.inf

for i in range(1, 10000):
  regr = Ridge(alpha=i)
  regr.fit(X_train, y_train)
  acc = mse(regr.predict(X_test), y_test)
  if acc < min_error:
    min_error = acc
    best_model = regr

print('best model parameters: ', best_model.get_params())
print('minimum error on test set: ',min_error)

best model parameters:  {'alpha': 167, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': 'deprecated', 'positive': False, 'random_state': None, 'solver': 'auto', 'tol': 0.001}
minimum error on test set:  14465.68633911328


In the above code, we see that the model with alpha = 167 has the best performance with 14465.67 mse error which is less than closed form linear regression model.

also here we used RidgeCV function in sklearn.linear_model package that uses cross-validation for setting its alpha parameter.

In [13]:
from sklearn.linear_model import RidgeCV

regr = RidgeCV(cv = 5, alphas = list(range(1, 2000))).fit(X_train, y_train)
acc = mse(regr.predict(X_test), y_test)
print(acc)


15165.470702293294


## 3. Lasso regression

here we use lasso regression for our model to predict the weights values. we use the scikit learn Lasso package and we test our model on different alpha values and we obtain the model with best accuracy (minimum mse error) on test set and we compare the results with Ridge regression model. we know that lasso regression loss function is:

$$L(w) = \frac{1}{n}\sum_{i=1}^{n}(y+i - X_iw)^2 + alpha * \sum_{j=1}^{m}\lvert w_j \rvert$$

In [14]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error as mse
best_model = None
min_error = math.inf

for i in range(1, 100):
  regr = Lasso(alpha=i, max_iter = 10000)
  regr.fit(X_train, y_train)
  acc = mse(regr.predict(X_test), y_test)
  if acc < min_error:
    min_error = acc
    best_model = regr

print('best model parameters: ', best_model.get_params())
print('minimum error on test set: ',min_error)

best model parameters:  {'alpha': 28, 'copy_X': True, 'fit_intercept': True, 'max_iter': 10000, 'normalize': 'deprecated', 'positive': False, 'precompute': False, 'random_state': None, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}
minimum error on test set:  14513.670665574979


In the above result, we see that minimum mse error is a bit more than the best ridge error and its near to the ridge result. we can also train lasso regression with cross validation using LassoCV function.

In [15]:
from sklearn.linear_model import LassoCV

regr = LassoCV(cv = 5, alphas = list(range(1, 100)), max_iter=10000).fit(X_train, y_train)
acc = mse(regr.predict(X_test), y_test)
print(acc)


14972.495168801703


## 4. elastic net regression
here we implemented the elastic net regression and we use this model for predicting. also we can convert this model to ridge regression model when we set the l_ratio parameter equal to zero and we can obtain lasso regression when we set the l_ratio parameter equal to one. here we optimize the loss function of this model by using the gradient method. the loss function for elastic net regression is:

$$L(w) = \frac{1}{n}\sum_{i=1}^{n}(y_i - X_iw)^2 + alpha * l * \sum_{j=1}^{m} \lvert w_j \rvert + l * 0.5 * (1 - alpha) * \sum_{j=1}^{m} w_j^2$$

**hint**: in this implementation, before using the model you must add one column of ones to train set for getting the predictions.

In [16]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error as mse
best_model = None
min_error = math.inf

for i in range(1, 1000):
  for j in range(1, 99):
    regr = ElasticNet(alpha=i, l1_ratio= j/100, max_iter = 10000)
    regr.fit(X_train, y_train)
    acc = mse(regr.predict(X_test), y_test)
    if acc < min_error:
      min_error = acc
      best_model = regr

print('best model parameters: ', best_model.get_params())
print('minimum error on test set: ',min_error)

best model parameters:  {'alpha': 7, 'copy_X': True, 'fit_intercept': True, 'l1_ratio': 0.87, 'max_iter': 10000, 'normalize': 'deprecated', 'positive': False, 'precompute': False, 'random_state': None, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}
minimum error on test set:  14434.058103846255


In the above result, we can see that ElasticNet best result is a bit lower (better) than Ridge regression and its value is very near to Ridge regression best model mse error. akso here we can train elastic net with cross validation using ElasticNetCV 

In [ ]:
from sklearn.linear_model import ElasticNetCV
best_model = None
min_error = math.inf

for i in range(1, 100):
  regr = ElasticNetCV(cv = 5, l1_ratio=i/100).fit(X_train, y_train)
  acc = mse(regr.predict(X_test), y_test)
  if acc < min_error:
    min_error = acc
    best_model = regr

print(min_error)


14439.755244981556


here, we can get the best model parameters that we achieved by using cross-validation.

In [18]:
best_model.alpha_

5.223482878455851

In [19]:
best_model.l1_ratio_

0.77

At the end, my opinion is that, linear regression is not a good model for predicting weights of the fish in this dataset!